In [ ]:
import os
import tweepy
from dotenv import load_dotenv

import json

In [ ]:
load_dotenv()

consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]
bearer_token = os.environ["BEARER_TOKEN"]


client = tweepy.Client(
  bearer_token=bearer_token,
  consumer_key=consumer_key,
  consumer_secret=consumer_secret,
  access_token=access_token,
  access_token_secret=access_token_secret,
  wait_on_rate_limit=True
)

In [ ]:
def extraer_cuentas_usuarios(lista_cuentas, lista_tweets, response):
    #for i in range(len(response.data)):
    for i in range(len(response.includes['users'])):
        #lista_cuentas.append(response.includes['users'][i].username)
        lista_cuentas.append(response.includes['users'][i])
        lista_tweets.append(response.data[i].text)

    print(len(lista_cuentas))

In [ ]:
# Search Recent Tweets
# This endpoint/method returns Tweets from the last seven days

query = 'depresión yo tengo'
#query = 'Petro'
#query = 'Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él'
#query = 'eafit'

# The method returns a Response object, a named tuple with data, includes,
# errors, and meta fields

cuentas_usuarios = []
tweets = []
for response in tweepy.Paginator(client.search_recent_tweets,
                                query,
                                tweet_fields=['id', 'author_id', 'text'],
                                user_fields=['id', 'username', 'location'],
                                expansions='author_id',
                                max_results=100, limit=1):#.flatten(limit=100):
    extraer_cuentas_usuarios(cuentas_usuarios, tweets, response)



In [ ]:
print(len(cuentas_usuarios))

In [ ]:
print(tweets)

In [ ]:
for usuario in cuentas_usuarios:
    print(usuario.location)

In [ ]:
usuarios = []
ids_usuarios = []
for usuario in cuentas_usuarios:
    if (usuario.location is not None):
        #if ('Medellin' in usuario.location) or ('Medellín' in usuario.location) or ('medellin' in usuario.location) or ('medellín' in usuario.location):
        if ('Colombia' in usuario.location) or ('colombia' in usuario.location):
            usuarios.append(usuario.username)
            ids_usuarios.append(usuario.id)

In [ ]:
print(usuarios)
print(ids_usuarios)

In [ ]:
diccionario_ids_usuarios = dict(zip(ids_usuarios, usuarios))

In [ ]:
def extraer_tweets_usuarios(id, diccionario, response):
    if id in list(diccionario.keys()):
        lista_tweets = diccionario[id]
    else:
        lista_tweets = []
    cont = 0
    for i in range(len(response.data)):
        if response.data[i]['referenced_tweets']:
            try:
                lista_tweets.append(response.includes['tweets'][cont].text)
                cont += 1
            except:
                lista_tweets.append(response.data[i].text)

        else:
            lista_tweets.append(response.data[i].text)

    diccionario[id] = lista_tweets

In [ ]:
users_tweets = dict()
for id_usuario in ids_usuarios:
    for response in tweepy.Paginator(client.get_users_tweets,
                                    id_usuario,
                                    tweet_fields=['id', 'author_id', 'text'],
                                    user_fields=['id', 'username', 'location'],
                                    expansions='referenced_tweets.id',
                                    #expansions='author_id',
                                    max_results=100, limit=1):#.flatten(limit=100):
        extraer_tweets_usuarios(id_usuario, users_tweets, response)

In [ ]:
print(users_tweets.keys())

In [ ]:
print(list(users_tweets.keys())[0])
print(users_tweets[list(users_tweets.keys())[0]])

In [ ]:
print(len(users_tweets[list(users_tweets.keys())[0]]))

In [ ]:
with open("tweets.json", "w") as fp:
    json.dump(users_tweets,fp)

In [ ]:
with open("users.json", "w") as fp:
    json.dump(diccionario_ids_usuarios,fp)